In [1]:
from rnamodif.architectures.rodan_pretrained_modcaller import RodanPretrainedModcaller
from rnamodif.data_utils.dataloading2 import nanopore_datamodule
from rnamodif.data_utils.split_methods import get_kfold_splits, get_fullvalid_split, get_valid_portions
import pytorch_lightning as pl
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning.callbacks import ModelCheckpoint

model = RodanPretrainedModcaller(lr=1e-4, use_Ns_for_loss=False, weighted_loss=False)

dm = nanopore_datamodule(
    splits = get_kfold_splits(
        pos_exps=['m6a_novoa_1', 'm6a_novoa_2'],
        neg_exps=['UNM_novoa_1', 'UNM_novoa_2', 'UNM_novoa_short'], 
        total_k=5, 
        current_k=0
    )
    +get_fullvalid_split()(pos_files=['m6A_5_covid','m6A_10_covid','m6A_33_covid'], neg_files=['m6A_0_covid']),
    verbose=1, 
    batch_size=32, 
    valid_limit=1000, 
    workers=16,
    window=4096,
    normalization='rodan',
)

experiment_name = 'm6a_novoa_base_unweighted_noNloss'
checkpoint_callback = ModelCheckpoint(
    dirpath=f"/home/jovyan/RNAModif/rnamodif/checkpoints_pl/{experiment_name}", 
    save_top_k=2, 
    monitor="valid_loss", 
    save_last=True, 
    save_weights_only=False
)

logger = CometLogger(api_key="TEVQbgxxvilM1WdTyqZLJ57ac", project_name='RNAModif', experiment_name=experiment_name) 
trainer= pl.Trainer(
    max_steps = 1000000, logger=logger, accelerator='gpu',
    auto_lr_find=False, val_check_interval=500,  
    log_every_n_steps=500, benchmark=True, precision=16,#)#,
    callbacks=[checkpoint_callback])
    # resume_from_checkpoint='/home/jovyan/RNAModif/rnamodif/checkpoints_pl/m6a_novoa_base_withNloss/last-v1.ckpt')


trainer.fit(model, dm)

CometLogger will be initialized in online mode
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:616: UserWarning: Checkpoint directory /home/jovyan/RNAModif/rnamodif/checkpoints_pl/m6a_novoa_base_unweighted_noNloss exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type                  | Params
----------------------------------------------------------
0 | trainable_rodan | network               | 10.7 M
1 | original_rodan  | network               | 10.7 M
2 | mod_neuron      | Linear                | 769   
3 | acc             | BinaryAccuracy        | 0     
4 | cm              | BinaryConfusionMatrix | 

Sanity Checking: 0it [00:00, ?it/s]

/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:219: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:86: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 32. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/torch/nn/functional.py:2904: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math

torch.Size([420, 32])


Exception: 